In [1]:
import pandas as pd
import gzip
import json

In [2]:
# Load the reviews
# 'lines=True' is required for this specific Amazon dataset format
df_reviews = pd.read_json('data/Electronics_5.json.gz', lines=True, compression='gzip')

# Check the size and the first few rows
print(f"Dataset Shape: {df_reviews.shape}")
df_reviews.head()

Dataset Shape: (6739590, 12)


,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,image
0,5,67,True,"09 18, 1999",AAP7PPBU72QFM,0151004714,{'Format:': ' Hardcover'},D. C. Carrad,This is the best novel I have read in 2 or 3 y...,A star is born,937612800,NaN
1,3,5,True,"10 23, 2013",A2E168DTVGE6SV,0151004714,{'Format:': ' Kindle Edition'},Evy,"Pages and pages of introspection, in the style...",A stream of consciousness novel,1382486400,NaN
2,5,4,False,"09 2, 2008",A1ER5AYS3FQ9O3,0151004714,{'Format:': ' Paperback'},Kcorn,This is the kind of novel to read when you hav...,I'm a huge fan of the author and this one did ...,1220313600,NaN
3,5,13,False,"09 4, 2000",A1T17LMQABMBN5,0151004714,{'Format:': ' Hardcover'},Caf Girl Writes,What gorgeous language! What an incredible wri...,The most beautiful book I have ever read!,968025600,NaN
4,3,8,True,"02 4, 2000",A3QHJ0FXK33OBE,0151004714,{'Format:': ' Hardcover'},W. Shane Schmidt,I was taken in by reviews that compared this b...,A dissenting view--In part.,949622400,NaN


In [3]:
# 1. Quick overview of data types and missing values
df_reviews.info()

# 3. Scale of the dataset
print(f"\nTotal Reviews: {len(df_reviews):,}")
print(f"Unique Products (ASINs): {df_reviews['asin'].nunique():,}")

<class 'pandas.DataFrame'>
RangeIndex: 6739590 entries, 0 to 6739589
Data columns (total 12 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   overall         int64 
 1   vote            str   
 2   verified        bool  
 3   reviewTime      str   
 4   reviewerID      str   
 5   asin            str   
 6   style           object
 7   reviewerName    str   
 8   reviewText      str   
 9   summary         str   
 10  unixReviewTime  int64 
 11  image           object
dtypes: bool(1), int64(2), object(2), str(7)
memory usage: 3.2+ GB

Total Reviews: 6,739,590
Unique Products (ASINs): 160,052


In [4]:
# 2. Distribution of ratings (as percentages)
round(df_reviews['overall'].value_counts(normalize=True).sort_index() * 100,2)

overall
1     6.93
2     4.55
3     7.49
4    16.88
5    64.15
Name: proportion, dtype: float64

In [5]:
# 1. Keep only the columns we need (saves a lot of RAM)
df_reviews = df_reviews[['asin', 'overall', 'reviewText', 'summary', 'unixReviewTime']]

# 2. Drop rows with empty reviews
df_reviews = df_reviews.dropna(subset=['reviewText'])

# 3. Take a 500,000 review sample
df_dev = df_reviews.sample(n=500000, random_state=42).copy()

# 4. Quick check: How many unique products are in our 500k sample?
print(f"Reviews in Dev Set: {len(df_dev):,}")
print(f"Unique Products in Dev Set: {df_dev['asin'].nunique():,}")

Reviews in Dev Set: 500,000
Unique Products in Dev Set: 103,816


In [6]:

# 1) Unique ASINs from the 500k dev set
sample_asins = set(df_dev['asin'].unique())
print(f"Unique ASINs in dev set: {len(sample_asins):,}")

# 2) Stream metadata and keep only rows for these ASINs
meta_rows = []

with gzip.open('data/meta_Electronics.json.gz', 'rt', encoding='utf-8') as f:
    for line in f:
        obj = json.loads(line)
        asin = obj.get('asin')
        if asin in sample_asins:
            meta_rows.append({
                'asin': asin,
                'title': obj.get('title'),
                'brand': obj.get('brand'),
                'category': obj.get('category'),
                'price': obj.get('price'),
            })

df_meta = pd.DataFrame(meta_rows)
print(f"Metadata rows kept: {len(df_meta):,}")
df_meta.head()

Unique ASINs in dev set: 103,816
Metadata rows kept: 108,382


,asin,title,brand,category,price
0,0101635370,abcGoodefg&reg; 4GB USB 2.0 Mp3 Music Player w...,Crazy Cart,"[Electronics, Portable Audio & Video, MP3 & MP...",
1,0151004714,The Last Life: A Novel,Visit Amazon's Claire Messud Page,"[Electronics, eBook Readers & Accessories, eBo...",$13.81
2,0446697192,Hollywood Is like High School with Money,Visit Amazon's Zoey Dean Page,"[Electronics, eBook Readers &amp; Accessories,...",$17.99
3,0511189877,CLIKR-5 Time Warner Cable Remote Control UR5U-...,URC,"[Electronics, Accessories &amp; Supplies, Audi...",
4,0528881469,Rand McNally 528881469 7-inch Intelliroute TND...,Rand McNally,"[Electronics, GPS, Finders & Accessories, Vehi...",


In [7]:
df_meta.info()

<class 'pandas.DataFrame'>
RangeIndex: 108382 entries, 0 to 108381
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   asin      108382 non-null  str   
 1   title     108382 non-null  str   
 2   brand     108382 non-null  str   
 3   category  108382 non-null  object
 4   price     108382 non-null  str   
dtypes: object(1), str(4)
memory usage: 20.1+ MB


In [9]:
# 1. Check how many duplicates we have
print(f"Metadata rows before deduplication: {len(df_meta):,}")
print(f"Unique ASINs in metadata: {df_meta['asin'].nunique():,}")

# 2. Keep only the FIRST entry for each ASIN
df_meta_unique = df_meta.drop_duplicates(subset=['asin'], keep='first')

print(f"Metadata rows after deduplication: {len(df_meta_unique):,}")

# 3. Re-run the join with the UNIQUE metadata
df_final = pd.merge(df_dev, df_meta_unique, on='asin', how='left')

print(f"Final Dataset Shape: {df_final.shape}")

Metadata rows before deduplication: 108,382
Unique ASINs in metadata: 103,732
Metadata rows after deduplication: 103,732
Final Dataset Shape: (500000, 9)


In [10]:
df_final.info()

<class 'pandas.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   asin            500000 non-null  str   
 1   overall         500000 non-null  int64 
 2   reviewText      500000 non-null  str   
 3   summary         499950 non-null  str   
 4   unixReviewTime  500000 non-null  int64 
 5   title           499492 non-null  str   
 6   brand           499492 non-null  str   
 7   category        499492 non-null  object
 8   price           499492 non-null  str   
dtypes: int64(2), object(1), str(6)
memory usage: 288.6+ MB


In [ ]:
# 1. Drop rows where metadata is missing (title, brand, or category)
df_final = df_final.dropna(subset=['title', 'brand', 'category'])

# 2. Fill the 50 missing 'summary' values (since the review text is still good)
df_final['summary'] = df_final['summary'].fillna("No Summary")

# 3. Final Verification
print(f"Final Dataset Size: {len(df_final):,}")
print("\nMissing Values Check:")
print(df_final.isnull().sum())

Final Dataset Size: 499,492

Missing Values Check:
asin              0
overall           0
reviewText        0
summary           0
unixReviewTime    0
title             0
brand             0
category          0
price             0
dtype: int64

✅ Phase 2 Complete: 100% Clean Dataset Saved.


In [16]:
df_final.info()

<class 'pandas.DataFrame'>
Index: 499492 entries, 0 to 499999
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   asin            499492 non-null  str   
 1   overall         499492 non-null  int64 
 2   reviewText      499492 non-null  str   
 3   summary         499492 non-null  str   
 4   unixReviewTime  499492 non-null  int64 
 5   title           499492 non-null  str   
 6   brand           499492 non-null  str   
 7   category        499492 non-null  object
 8   price           499492 non-null  str   
dtypes: int64(2), object(1), str(6)
memory usage: 292.1+ MB


In [20]:
# 1. Calculate word count for each review
df_final['word_count'] = df_final['reviewText'].str.split().str.len()

# 2. See the distribution
print("Review Length Distribution:")
print(df_final['word_count'].describe())

# 3. Check how many are "very short" (e.g., < 5 words like "Great product!")
short_reviews = (df_final['word_count'] < 5).sum()
print(f"\nReviews with < 5 words: {short_reviews:,} ({short_reviews/len(df_final)*100:.1f}%)")

# 4. Check how many are "very long" (e.g., > 200 words like "Great product!")
long_reviews = (df_final['word_count'] > 200).sum()
print(f"\nReviews with > 200 words: {long_reviews:,} ({long_reviews/len(df_final)*100:.1f}%)")

Review Length Distribution:
count    499492.000000
mean         65.541190
std         118.024363
min           1.000000
25%          10.000000
50%          29.000000
75%          72.000000
max        5682.000000
Name: word_count, dtype: float64

Reviews with < 5 words: 67,778 (13.6%)

Reviews with > 200 words: 34,983 (7.0%)


In [21]:
# 1. Keep only reviews with 10 or more words (the "High Signal" filter)
df_final = df_final[df_final['word_count'] >= 10].copy()

# 2. Drop the 'word_count' column
df_final = df_final.drop(columns=['word_count'])

# 3. Final Verification
print(f"Final Dataset Size: {len(df_final):,}")

# 4. Save the "Platinum Standard" file
df_final.to_parquet('data/reviews_electronics_500k.parquet', index=False)
print("\n✅ Phase 2 COMPLETE. You have a high-signal dataset ready for AI.")

Final Dataset Size: 379,019

✅ Phase 2 COMPLETE. You have a high-signal dataset ready for AI.
